In [ ]:
##### DEFINE FUNCTIONS #####

In [ ]:
## Get anomalies/composites
# Define anomaly function
def get_anomaly(x,detrend):
    clim = x.groupby('time.month').mean(dim='time')
    if detrend == 1:
        poly = x.polyfit(dim='time', deg=1)
        fit = xr.polyval(x['time'], poly.polyfit_coefficients)
        return x.groupby('time.month') - clim - fit
    else:
        return x.groupby('time.month') - clim

# Define composite functions
def composite(x):
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom_lo_res/nino3_4/'
    nino34 = xr.open_dataset(path+'nino34_index.nc').__xarray_dataarray_variable__
    
    return x.groupby(nino34.rename('nino34')).mean(dim='time')

In [ ]:
## Get global-means
def global_mean(x):
    return x.weighted(weight).mean(dim=('latitude','longitude'))

In [ ]:
## Get Clear Sky Radiation Anomalies across pressure levels
def get_mean_across_lev(kernel,del_param):
    arr = []
    kernel = kernel.transpose('plev',...)
    del_param = del_param.transpose('plev',...)
    
    for i in range(0,len(kernel.plev)):           
        # Do calculation at each p level
        if i != len(kernel.plev)-1:
            calc = kernel[i] * del_param[i] * (kernel.plev[i] - kernel.plev[i+1])/100
        else:
            calc = kernel[i] * del_param[i] * (kernel.plev[i] - 0)/100
        
        # Append to arrays
        arr.append(calc)
    
    # Concat each array into DataArrays, and then sum
    calc_da = xr.concat(arr, dim='plev')
    calc_xr = calc_da.sum(dim='plev')
    calc_mon_avg = calc_xr.mean(dim='month')
    
    return calc_mon_avg

In [ ]:
## Get CRE across p,tau bins
def get_mean_across_bins(x_cld,x_clr,cld_area):
    arr = []
    n = int(x_cld.press[0])
    m = int(x_cld.press[-1])
    
    for i in range(n,m):
        for j in range(0,6):            
            # Do calculation in each p,tau bin
            calc = cld_area.sel(press=i,opt=j) * (x_cld.sel(press=i,opt=j) - x_clr)
            
            # Append to arrays
            arr.append(calc)
    
    # Concat each array into DataArrays, and then sum
    calc_da = xr.concat(arr, dim='pressopt')
    calc_xr = calc_da.sum(dim='pressopt')
    
    return calc_xr

In [ ]:
## Get low cloud and high cloud cre from FBCT
def calc_fbct_cre(lw_flux_cld,lw_flux_clr,sw_flux_cld,sw_flux_clr,cld_frac):
    # Get net DOWNWARD flux and then the climatological mean
    net_flux_all = (-sw_flux_cld - lw_flux_cld).mean(dim='time')
    net_flux_clr = (-sw_flux_clr - lw_flux_clr).mean(dim='time')

    # Downsample the net DOWNWARD flux
    net_flux_all_loRes = net_flux_all.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    net_flux_clr_loRes = net_flux_clr.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    
    # Downsample the cloud fraction
    cld_frac_loRes = cld_frac.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    cld_frac_anom = get_anomaly(cld_frac_loRes,1)
    
    # Finally, sum across p,tau bins to get monthly cloud radiative anomaly *** (R_pt - R_clr)*f'_pt ***
    get_cre = get_mean_across_bins(net_flux_all_loRes,net_flux_clr_loRes,cld_frac_anom)
    
    return get_cre

In [ ]:
## Get net cre, clear-sky, and all-sky from EBAF
def calc_net_cre():
    net_flux_cld = EBAF_flux.toa_net_all_mon
    net_flux_clr = EBAF_flux.toa_net_clr_t_mon
    
    net_flux_cld_loRes = net_flux_cld.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    net_flux_clr_loRes = net_flux_clr.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    
    net_cre_loRes = net_flux_cld_loRes - net_flux_clr_loRes
    calc_net = get_anomaly(net_cre_loRes,1)
    
    return calc_net

def calc_ebaf_rad_anom(data):
    ebaf_rad_anom = get_anomaly(data,1)
    ebaf_rad_anom_loRes = ebaf_rad_anom.interp(latitude=new_lat_medRes,longitude=new_lon_medRes)
    
    return ebaf_rad_anom_loRes